In [65]:
from fn import op, _, F
from operator import add, mul
from itertools import repeat, chain
from functools import reduce
# from objproxies import LazyProxy
import json
from copy import deepcopy, copy
from pipetools import pipe
from functools import partial
import numpy as np
from datetime import datetime, timedelta
from decimal import Decimal, getcontext, ROUND_DOWN
TWOPLACES = Decimal(10) ** -2       # same as Decimal('0.01')
import pandas as pd
# getcontext().prec=None
# getcontext().rounding = ROUND_DOWN

In [14]:
flatten = lambda l: [item for sublist in l for item in sublist]

def flatmap(f, items):
        return list(map(f, items))
    
def last_index(l):
    return len(l)-1

def retrieve_state(l, offset):
    return l[last_index(l) + offset + 1]
# Stochastic Process
# Input RNG Seed
def bound_norm_random(rng, low, high):
    # Add RNG Seed
    res = rng.normal((high+low)/2,(high-low)/6)
    if (res<low or res>high):
        res = bound_norm_random(rng, low, high)
    return Decimal(res)

def env_proc(trigger_step, update_f):
    def env_step_trigger(trigger_step, update_f, step):
        if step == trigger_step:
            return update_f
        else:
            return lambda x: x
    return partial(env_step_trigger, trigger_step, update_f)

def time_step(s, dt_str, fromat_str='%Y-%m-%d %H:%M:%S', days=0, minutes=0, seconds=1):
    if s['mech_step'] == 0:
        dt = datetime.strptime(dt_str, fromat_str)
        t = dt + timedelta(days=days, minutes=minutes, seconds=seconds)
        return t.strftime(fromat_str)
    else: 
        return dt_str

def round_down(x, fp=TWOPLACES):
    return x.quantize(TWOPLACES, rounding=ROUND_DOWN)
    
# def round_down(f, fp=TWOPLACES):
#     return (f).quantize(fp, ROUND_DOWN)

In [15]:

# esceptions:
# point to line giving problems and pass a sentinel

In [16]:
seed = {
    'z': np.random.RandomState(1),
    'a': np.random.RandomState(2),
    'b': np.random.RandomState(3),
    'c': np.random.RandomState(3)
}

# UI Behaviors per Mechanism
def b1m1(step, sL, s):
    return s['s1']
def b2m1(step, sL, s):
    return s['s1'] * s['s2']

def b1m2(step, sL, s):
    return s['s1']
def b2m2(step, sL, s):
    return s['s1'] / s['s2']

def b1m3(step, sL, s):
    return s['s1'] + seed['z'].randint(1000)
def b2m3(step, sL, s):
    ps = retrieve_state(sL, -3)
    return ps['s2']

# UI Internal States per Mechanism
def s1m1(step, sL, s, _input):
#     print(s['s1'])
    s['s1'] = s['s1'] + _input
def s2m1(step, sL, s, _input):
    s['s2'] = s['s2'] + 1 + _input

def s1m2(step, sL, s, _input):
    s['s1'] = s['s1'] + _input
def s2m2(step, sL, s, _input):
    s['s2'] = s['s2']
    
def s1m3(step, sL, s, _input):
    s['s1'] = s['s1']
def s2m3(step, sL, s, _input):
    s['s2'] = s['s2'] + _input

# UI Exogenous States #per Mechanism
proc_one_coef_A = 0.7
proc_one_coef_B = 1.3
def es3p1(step, sL, s, _input):
    s['s3'] = s['s3'] * bound_norm_random(seed['a'], proc_one_coef_A, proc_one_coef_B)
def es4p2(step, sL, s, _input):
    s['s4'] = s['s4'] * bound_norm_random(seed['b'], proc_one_coef_A, proc_one_coef_B)  
def es5p2(step, sL, s, _input):
    s['timestamp'] = time_step(s, s['timestamp'], seconds=1)


# add env process f(s) read from time es
# funcs execute in order
def env_a(x): 
    return x + 1
def env_b(x): 
    return x + 2
    
def what_ever(x): 
    return x + 1

# Genesis States    
state_dict = {
    's1': Decimal(2.0),
    's2': Decimal(4.0),
    's3': Decimal(0.0),
    's4': Decimal(0.0),
    'timestamp': '2018-10-01 15:16:24'
}

exogenous_states = {
    "s3": es3p1,
    "s4": es4p2,
    "timestamp": es5p2
}

# Update time at the end of each pipeline (once per 'block')
# Enable the acceptance of user defined time
env_processes = {
    "s3": env_proc(1, env_a),
    "s4": env_proc(1, pipe | env_b | what_ever)
}
    
mechanisms = {
    "m1": {
        "behaviors": {
            "b1": b1m1,
            "b2": b2m1
        },
        "states": {
            "s1": s1m1,
            "s2": s2m1,
        }
    },
    "m2": {
        "behaviors": {
            "b1": b1m2,
            "b2": b2m2
        },
        "states": {
            "s1": s1m2,
            "s2": s2m2,
        }
    },
    "m3": {
        "behaviors": {
            "b1": b1m3,
            "b2": b2m3
        },
        "states": {
            "s1": s1m3,
            "s2": s2m3,
        }
    }
}

In [73]:
# if beh list empty, repeat 0 x n_states in list 
def generate_config(mechanisms, exogenous_states):
    es_funcs = [exogenous_states[state] for state in list(exogenous_states.keys())]
    config = list(
        map(
            lambda m: (
                list(mechanisms[m]["states"].values()) + es_funcs,
                list(mechanisms[m]["behaviors"].values())
            ), 
            list(mechanisms.keys())
        )
    )
    return config

# partials = list(map(lambda f: partial(f, step, sL), funcs))
def getColResults(step, sL, s, funcs):
    return list(map(lambda f: f(step, sL, s), funcs))

def getBehaviorInput(step, sL, s, funcs): 
    return op.foldr(_ + _)(getColResults(step, sL, s, funcs))

def apply_env_proc(env_processes, state_dict, step):
    for state in state_dict.keys(): 
        if state in list(env_processes.keys()):
            state_dict[state] = env_processes[state](step)(state_dict[state])
#     return state_dict
            
def mech_step(m_step, sL, state_funcs, behavior_funcs, env_processes, t_step):
    # Purge Memory accociated with certain objs
    # Truncate reffs
    in_copy, mutatable_copy, out_copy = deepcopy(sL), deepcopy(sL), deepcopy(sL)
    last_in_obj, last_mut_obj = in_copy[-1], mutatable_copy[-1]
    
    # * force eval of _input withing state functions
    _input = getBehaviorInput(m_step, sL, last_in_obj, behavior_funcs)

    apply_env_proc(env_processes, last_mut_obj, t_step)
    
#     print(last_mut_obj)
    
#     non_numeric = {k: v for k, v in last_mut_obj.items() if type(v) != Decimal}
#     last_mut_obj = {k: round_down(v) for k, v in last_mut_obj.items() if type(v) == Decimal}
#     last_mut_obj.update(non_numeric)

    for f in state_funcs:
        f(m_step, sL, last_mut_obj, _input)
        
    last_mut_obj["mech_step"], last_mut_obj["time_step"] = m_step, t_step
    out_copy.append(last_mut_obj)
    
    del last_in_obj, last_mut_obj, in_copy, mutatable_copy,
    return out_copy

def block_gen(states_list, configs, env_processes, t_step):
    m_step = 0
    states_list_copy = deepcopy(states_list)
    genesis_states = states_list_copy[-1]
    genesis_states['mech_step'], genesis_states['time_step'] = m_step, t_step
    states_list = [genesis_states]
    
    m_step += 1
    for config in configs:
        s_conf, b_conf = config[0], config[1]
        states_list = mech_step(m_step, states_list, s_conf, b_conf, env_processes, t_step)     
        m_step += 1
        
    t_step += 1    
    
    return states_list

def pipeline(states_list, configs, env_processes, time_seq):
    time_seq = [x + 1 for x in time_seq]
    simulation_list = [states_list]
    for time_step in time_seq:
        pipeline_run = block_gen(simulation_list[-1], configs, env_processes, time_step)
        head, *pipeline_run = pipeline_run
        simulation_list.append(pipeline_run)
    
    # should only occur once for genesis state, placed before loop
    head, *tail = simulation_list
    head[-1]['mech_step'], head[-1]['time_step'] = 0, 0
    simulation_list = [head] + tail
    return simulation_list

In [31]:
states_list[-1]

{'s1': Decimal('2'),
 's2': Decimal('4'),
 's3': Decimal('0'),
 's4': Decimal('0'),
 'timestamp': '2018-10-01 15:16:24',
 'mech_step': 0,
 'time_step': 0}

In [71]:
flatten(pipeline(states_list, configs, env_processes, range(10)))

"[{'s1': Decimal('2'), 's2': Decimal('4'), 's3': Decimal('0'), 's4': Decimal('0'), 'timestamp': '2018-10-01 15:16:24', 'mech_step': 0, 'time_step': 0}, {'s1': Decimal('12'), 's2': Decimal('15'), 's3': Decimal('1.027943375654487123682656602'), 's4': Decimal('2.873162181767157652423350100'), 'timestamp': '2018-10-01 15:16:25', 'mech_step': 1, 'time_step': 1}, {'s1': Decimal('24.8'), 's2': Decimal('15'), 's3': Decimal('1.658405192679980226295085390'), 's4': Decimal('5.350515951995874829597979645'), 'timestamp': '2018-10-01 15:16:25', 'mech_step': 2, 'time_step': 1}, {'s1': Decimal('24.8'), 's2': Decimal('513.8'), 's3': Decimal('2.771923735333028371712141099'), 's4': Decimal('7.030344993582647198055332758'), 'timestamp': '2018-10-01 15:16:25', 'mech_step': 3, 'time_step': 1}, {'s1': Decimal('12791.84'), 's2': Decimal('13281.84'), 's3': Decimal('2.884737635508387705829249992'), 's4': Decimal('6.383750387315317896150130866'), 'timestamp': '2018-10-01 15:16:26', 'mech_step': 1, 'time_step': 2

In [75]:
states_list = [state_dict]
configs = generate_config(mechanisms, exogenous_states)
# type(pipeline(states_list, configs, env_processes, range(10)))
pd.DataFrame(flatten(pipeline(states_list, configs, env_processes, range(10))))

,mech_step,s1,s2,s3,s4,time_step,timestamp
0,0,2,4,0,0,0,2018-10-01 15:16:24
1,1,12,15,0.9134790035783337813768412161,2.502245379381494383430606376,1,2018-10-01 15:16:25
2,2,24.8,15,2.248685730584553218869219712,5.512870077468841289396533774,1,2018-10-01 15:16:25
3,3,24.8,548.8,3.431275232953615539312717484,9.152222843429326282106045717,1,2018-10-01 15:16:25
4,1,13659.84,14184.84,3.322814672054344544572535039,7.708815382252544561603904880,2,2018-10-01 15:16:26
5,2,27320.64298865549417547184177,14184.84,2.897641633315681179441300144,7.871440379399112665270434205,2,2018-10-01 15:16:26
6,3,27320.64298865549417547184177,43033.28298865549417547184177,3.109937351467413775751721194,8.630776907929052893315997214,2,2018-10-01 15:16:26
7,1,1175751602.448815787831625727,1175767316.088815787831625727,3.176228364900735157136598597,9.476875554027664732983490933,3,2018-10-01 15:16:27
8,2,2351503205.897618211080153268,1175767316.088815787831625727,3.669342085393265738584747393,9.078969322902328500177642109,3,2018-10-01 15:16:27
9,3,2351503205.897618211080153268,3527313580.269422654405954467,3.349759960987064464357988348,11.17144391140400526366105626,3,2018-10-01 15:16:27


In [42]:
for x in list(range(5)):
    print(x)

0
1
2
3
4


In [61]:
def simulation(states_list, configs, env_processes, time_seq, runs):
    pipeline_run = []
    for run in range(runs):
        if run == 0:
            pipeline_run += pipeline(states_list, configs, env_processes, time_seq)
#             pipeline_run += [run]
        else:
            pipeline_run += pipeline(pipeline_run[-1], configs, env_processes, time_seq)
#             pipeline_run += [run]
    return pipeline_run

In [79]:
list(range(2))

[0, 1]

In [77]:
pd.DataFrame(flatten(simulation(states_list, configs, env_processes, range(2), 2)))

,mech_step,s1,s2,s3,s4,time_step,timestamp
0,0,2,4,0,0,0,2018-10-01 15:16:24
1,1,12,15,0.9613374219112287288879770131,3.052627916233361826670034134,1,2018-10-01 15:16:25
2,2,24.8,15,1.787841886705475247843978831,6.142108075626036207691665842,1,2018-10-01 15:16:25
3,3,24.8,463.8,2.660351416005110429442387343,9.305727043596993943746695376,1,2018-10-01 15:16:25
4,1,11551.84,11991.84,2.464645185343364655440715846,9.337777284373435717315011919,2,2018-10-01 15:16:26
5,2,23104.64330838303379631482742,11991.84,2.304639148261938062624317980,10.29602467538929243745549665,2,2018-10-01 15:16:26
6,0,23104.64330838303379631482742,36429.28330838303379631482742,1.745014350171677143026393926,10.79831922713935264930519165,0,2018-10-01 15:16:26
7,1,11551.84,11991.84,2.464645185343364655440715846,9.337777284373435717315011919,2,2018-10-01 15:16:26
8,2,23104.64330838303379631482742,11991.84,2.304639148261938062624317980,10.29602467538929243745549665,2,2018-10-01 15:16:26
9,0,23104.64330838303379631482742,36429.28330838303379631482742,1.745014350171677143026393926,10.79831922713935264930519165,0,2018-10-01 15:16:26


In [7]:
# Decimal('240.0').quantize(Decimal('0.01'), ROUND_DOWN)
# round(Decimal('240.01'), 2)
state_dict = {'s1': Decimal('3323'), 's2': Decimal('2572'), 's3': Decimal('2.657'), 's4': Decimal('7.914'), 'timestamp': '2018-10-01 15:16:26', 'mech_step': 1, 'time_step': 2}
state_dict = {k: v.quantize(TWOPLACES, ROUND_DOWN) for k, v in state_dict.items() if type(v) == Decimal}
state_dict.update(non_numeric)
state_dict

NameError: name 'non_numeric' is not defined

In [ ]:
states_list = [state_dict]
states_list
# {'s1': 8.742503764992283e+170,
#    's2': 1.119720780594405e+171,
#    's3': 4.251811100418906,
#    's4': 6.92728042985628,
#    'timestamp': '2018-10-01 15:16:31',
#    'mech_step': 3,
#    'time_step': 7}
# {'s1': Decimal('2.221384236472411425810660220E+330'),
#    's2': Decimal('2.845096899831187764579913612E+330'),
#    's3': Decimal('3.622519693611211445932083740'),
#    's4': Decimal('5.161487081026327858663750787'),
#    'timestamp': '2018-10-01 15:16:32',
#    'mech_step': 3,
#    'time_step': 8}

In [ ]:
from decimal import Decimal, MAX_EMAX
# Decimal(2.5347751615574597e+167)**2
MAX_EMAX**5

In [ ]:
1000 * 1000

In [ ]:
from datetime import datetime, timedelta
# from datetime import timedelta  
# st = datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
st = datetime.now()
print(st)
x = st + timedelta(seconds=60)
print(x)

In [ ]:
s = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
datetime_object = datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
datetime_object

In [ ]:
datetime.fromtimestamp()

In [ ]:
from datetime import datetime, timedelta

# def time_step_func(year, month, day, hour, minute, second):
a = datetime(2016, 11, 15, 9, 59, 25)#.strftime("%Y-%m-%d %H:%M:%S")
b = a + timedelta(seconds=60) # days, seconds, then other fields.
print(a)
print(b)
st = datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
def time_step_func(dt_str, fromat_str='%Y-%m-%d %H:%M:%S', days=0, minutes=0, seconds=1):
    dt = datetime.strptime(dt_str, fromat_str)
    t = dt + timedelta(days=days, minutes=minutes, seconds=seconds)
    return t.strftime(fromat_str)

time_step_func('2018-10-01 15:16:24')

In [ ]:
def time_step_func(x, y):
    return x + y

In [ ]:
for m in M: #M is the parameter sweep array
    # for Pairwise, run below separately
    for n in N: #N is the total number of Montecarlo runs
        for t in T: #T[len(T)]-T[0] is the duration of each simulation
            # Pipeline
            for b in Behaviors:
                for M in Mechanisms:
                    for s in States:
                        update(s)

In [ ]:
[{'s1': 2}] + [{'s1': 3}]
list(zip([block_gen] * 2, range(2)))
len([0,1])

In [ ]:
states_list = [state_dict]
configs = generate_config(mechanisms, exogenous_states)
def step_func(x, y):
    return x + y
pipeline(states_list, configs, env_processes, step_func)

In [ ]:
# def s3step1(step, sL, s, _input):
#     s['s3'] = s['s3']    
# # def s3step2(step, sL, s, _input):
#     s['s3'] = s['s3'] + 1
# def s3step3(step, sL, s, _input):
#     s['s3'] = s['s3'] * bound_norm_random(proc_one_coef_A, proc_one_coef_B)

In [ ]:
# 15 + 28.933333333333334 + 4

In [ ]:
# def multiply(x,y,z):
#         return x * y * z

# # create a new function that multiplies by 2
# dbl = partial(partial(multiply,2), multiply, 2)
# print(dbl(2))